In [3]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = "google/pegasus-xsum"  # Choose appropriate PEGASUS model size
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import pandas as pd
#load data

df_train = pd.read_excel('D:/dataaa/Topic/df_train_topic_1.xlsx')
df_validation = pd.read_excel('D:/dataaa/Topic/df_validation_topic_1.xlsx')
df_test = pd.read_excel('D:/dataaa/Topic/df_test_topic_1.xlsx')

In [6]:
df_train.head(10)

,article,highlights,topic
0,"['fleetwood', 'team', 'still', '100', 'record'...","['fleetwood', 'top', 'league', 'one', '20', 'w...",1
1,"['number', 'job', 'description', 'waiting', 'd...","['tony', 'pulis', 'belief', 'saido', 'berahino...",1
2,"['louis', 'van', 'gaal', 'said', 'option', 'su...","['manchester', 'united', 'beat', 'southampton'...",1
3,"['everton', 'still', 'looking', 'add', 'two', ...","['everton', 'looking', 'add', 'two', 'new', 'p...",1
4,"['glen', 'johnson', 'look', 'destined', 'leave...","['full', 'back', 'scored', 'late', 'winner', '...",1
5,"['louis', 'van', 'gaal', 'welcomed', 'public',...","['manchester', 'united', 'host', 'newcastle', ...",1
6,"['chelseas', 'early', 'season', 'form', 'may',...","['chelsea', 'four', 'point', 'clear', 'top', '...",1
7,"['john', 'drayton', 'lionel', 'messi', 'took',...","['messi', 'led', 'argentina', 'team', 'talk', ...",1
8,"['say', 'lightning', 'doesnt', 'strike', 'twic...","['jar', 'jar', 'binks', 'chewbacca', 'r2d2', '...",1
9,"['germany', 'captain', 'philipp', 'lahm', 'cla...","['philipp', 'lahm', 'say', 'germany', 'become'...",1


In [7]:
def generate_summary(text, max_length=1024):
    # Truncate or split the text if it exceeds the model's maximum token limit
    tokens = tokenizer.encode(text, return_tensors="pt", truncation=True)
    tokens = tokens[:, :max_length]
    
    # Generate the summary
    output = model.generate(tokens, max_length=max_length, num_beams=4, length_penalty=2.0, early_stopping=True)
    
    # Decode the generated summary
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return summary


In [8]:
df_train['summary'] = df_train['article'].head(10).apply(generate_summary)

In [9]:
df_train.head(10)

,article,highlights,topic,summary
0,"['fleetwood', 'team', 'still', '100', 'record'...","['fleetwood', 'top', 'league', 'one', '20', 'w...",1,BBC Sport picks out the best quotes from the w...
1,"['number', 'job', 'description', 'waiting', 'd...","['tony', 'pulis', 'belief', 'saido', 'berahino...",1,"Number, 'job', description, 'waiting', 'darren..."
2,"['louis', 'van', 'gaal', 'said', 'option', 'su...","['manchester', 'united', 'beat', 'southampton'...",1,BBC Sport's football expert Mark Lawrenson loo...
3,"['everton', 'still', 'looking', 'add', 'two', ...","['everton', 'looking', 'add', 'two', 'new', 'p...",1,Match of the Day presenter Gary Lineker gives ...
4,"['glen', 'johnson', 'look', 'destined', 'leave...","['full', 'back', 'scored', 'late', 'winner', '...",1,Match of the Day presenter Gary Lineker gives ...
5,"['louis', 'van', 'gaal', 'welcomed', 'public',...","['manchester', 'united', 'host', 'newcastle', ...",1,BBC Sport's football expert Gary Lineker gives...
6,"['chelseas', 'early', 'season', 'form', 'may',...","['chelsea', 'four', 'point', 'clear', 'top', '...",1,"Chelsea, Arsenal, Manchester United, Mancheste..."
7,"['john', 'drayton', 'lionel', 'messi', 'took',...","['messi', 'led', 'argentina', 'team', 'talk', ...",1,The words'messi' and 'lionel' have been replac...
8,"['say', 'lightning', 'doesnt', 'strike', 'twic...","['jar', 'jar', 'binks', 'chewbacca', 'r2d2', '...",1,BBC Sport's football expert Mark Lawrenson giv...
9,"['germany', 'captain', 'philipp', 'lahm', 'cla...","['philipp', 'lahm', 'say', 'germany', 'become'...",1,Germany's World Cup-winning captain Philipp La...


In [8]:
output_test_excel_file = 'D:/dataaa/df_train_summary1.xlsx'
df_train.head(50).to_excel(output_test_excel_file, index=False)

In [10]:
#apply the Gpt-2

import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [11]:
# Function to generate summaries using GPT-2
def generate_summary(article):
    inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=149, truncation=True)
    inputs = inputs[:, :511]  # Adjust the size to match the model's input size

    # Generate summary with attention_mask and pad_token_id set
    summary_ids = model.generate(
        inputs,
        max_length=150,  # Adjust the max_length to control the length of the summary
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Handle potential output size discrepancies
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [12]:
# Apply the generate_summary function to the entire dataset
df_train['generated_summary'] = df_train['article'].head(10).apply(generate_summary)

In [13]:
df_train.head(10)

,article,highlights,topic,summary,generated_summary
0,"['fleetwood', 'team', 'still', '100', 'record'...","['fleetwood', 'top', 'league', 'one', '20', 'w...",1,BBC Sport picks out the best quotes from the w...,"summarize: ['fleetwood', 'team','still', '100'..."
1,"['number', 'job', 'description', 'waiting', 'd...","['tony', 'pulis', 'belief', 'saido', 'berahino...",1,"Number, 'job', description, 'waiting', 'darren...","summarize: ['number', 'job', 'description', 'w..."
2,"['louis', 'van', 'gaal', 'said', 'option', 'su...","['manchester', 'united', 'beat', 'southampton'...",1,BBC Sport's football expert Mark Lawrenson loo...,"summarize: ['louis', 'van', 'gaal','said', 'op..."
3,"['everton', 'still', 'looking', 'add', 'two', ...","['everton', 'looking', 'add', 'two', 'new', 'p...",1,Match of the Day presenter Gary Lineker gives ...,"summarize: ['everton','still', 'looking', 'add..."
4,"['glen', 'johnson', 'look', 'destined', 'leave...","['full', 'back', 'scored', 'late', 'winner', '...",1,Match of the Day presenter Gary Lineker gives ...,"summarize: ['glen', 'johnson', 'look', 'destin..."
5,"['louis', 'van', 'gaal', 'welcomed', 'public',...","['manchester', 'united', 'host', 'newcastle', ...",1,BBC Sport's football expert Gary Lineker gives...,"summarize: ['louis', 'van', 'gaal', 'welcomed'..."
6,"['chelseas', 'early', 'season', 'form', 'may',...","['chelsea', 'four', 'point', 'clear', 'top', '...",1,"Chelsea, Arsenal, Manchester United, Mancheste...","summarize: ['chelseas', 'early','season', 'for..."
7,"['john', 'drayton', 'lionel', 'messi', 'took',...","['messi', 'led', 'argentina', 'team', 'talk', ...",1,The words'messi' and 'lionel' have been replac...,"summarize: ['john', 'drayton', 'lionel','messi..."
8,"['say', 'lightning', 'doesnt', 'strike', 'twic...","['jar', 'jar', 'binks', 'chewbacca', 'r2d2', '...",1,BBC Sport's football expert Mark Lawrenson giv...,"summarize: ['say', 'lightning', 'doesnt','stri..."
9,"['germany', 'captain', 'philipp', 'lahm', 'cla...","['philipp', 'lahm', 'say', 'germany', 'become'...",1,Germany's World Cup-winning captain Philipp La...,"summarize: ['germany', 'captain', 'philipp', '..."
